In [1]:
#IMPORTS
import numpy as np
import pydicom as dicom
from pydicom.fileset import FileSet
import pathlib
import os
import glob
import shutil
#https://medium.com/p/f1623a7f40b8
#https://gist.githubusercontent.com/alex-weston-13/4dae048b423f1b4cb9828734a4ec8b83/raw/59e49d168421a667645c9ab231d3993fe0752724/sort_dicoms.py

In [3]:
# dicomfilepath = pathlib.PureWindowsPath('E:\\IFL\\digibiop\\0001211180\\DICOMDIR')
dicomfilepath_iflpc = pathlib.PurePosixPath('/mnt/projects/DeepProstateDB/Data/14/0001484378/DICOMDIR')
# ds = dicom.dcmread(dicomfilepath)
ds = dicom.dcmread(dicomfilepath_iflpc)  #ds stores the path to dicom directory
print(ds)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 156
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Media Storage Directory Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.840.10008.123456.691897.2016008521.4927756976.129280377
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1
(0002, 0013) Implementation Version Name         SH: ''
-------------------------------------------------
(0004, 1130) File-set ID                         CS: 'SECTRA FILE SET'
(0004, 1141) File-set Descriptor File ID         CS: ''
(0004, 1142) Specific Character Set of File-set  CS: 'ISO_IR 100'
(0004, 1200) Offset of the First Directory Recor UL: 396
(0004, 1202) Offset of the Last Directory Record UL: 396
(0004, 1212) File-set Consistency Flag           US: 0
(0004, 1220)  Directory Reco

In [ ]:
file = FileSet(ds)
# print(file)

In [3]:
#for windows
# src = "E:\\IFL\\digibiop\\0001211180\\DICOM\\0000D122\\AA06F95E\\AAF40D12"
# dst = "E:\\IFL\\digibiop\\Copies"

#for linux iflpc
src = "/mnt/projects/DeepProstateDB/Data/19/0002208807/DICOM/0000A326/AA1740DD/AA0ABBD7/0000A9DF"
dst = "/mnt/HDD1/shounak/test"
print('reading file list...')
unsortedList = []
for root, dirs, files in os.walk(src):
    print("root",root)
    print("dirs",dirs)
    print("files",files)
    for file in files: 
#         if ".file" in file:# exclude non-dicoms, good for messy folders
            unsortedList.append(os.path.join(root, file))

print('%s files found.' % len(unsortedList))

reading file list...
root /mnt/projects/DeepProstateDB/Data/19/0002208807/DICOM/0000A326/AA1740DD/AA0ABBD7/0000A9DF
dirs []
files ['EE0256DD', 'EE1034A6', 'EE19E628', 'EE1DB2B5', 'EE22A7C5', 'EE261EA7', 'EE2675B5', 'EE2DBF27', 'EE303988', 'EE30D06A', 'EE34CDDA', 'EE37AF29', 'EE393912', 'EE3E7D8B', 'EE40FE5F', 'EE457873', 'EE489BB7', 'EE4D202C', 'EE4DF2D9', 'EE60015C', 'EE62D5BC', 'EE63D37D', 'EE70C127', 'EE73BE4F', 'EE75DD64', 'EE7BF4C8', 'EE7E9186', 'EE8817C7', 'EE887BFE', 'EE897681', 'EE8A892D', 'EE91CA20', 'EE9678B3', 'EEA56C5A', 'EEAB7F31', 'EEABE0E6', 'EEAC5C08', 'EEAC9F42', 'EEAF8A5D', 'EEB07901', 'EEB5AE04', 'EEBFE456', 'EEC18658', 'EECE9540', 'EECF6369', 'EED36620', 'EED5B7B2', 'EEE01EB0', 'EEE0BC77', 'EEEC5DDB', 'EEF2EC95', 'EEFA53B8', 'EEFBAE8B', 'EEFF9339']
54 files found.


In [ ]:
testfile = dicom.dcmread('E:\\IFL\\digibiop\\0001211180\\DICOM\\0000D122\\AA06F95E\\AAF40D12\\0000E2BE\\EE0A8535')
print(testfile[0x0008, 0x103e])

In [44]:
#EXCLUDE PUNCTUATIONS

def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    return string.lower()

In [46]:
for dicom_loc in unsortedList:
    # read the file
    ds = dicom.read_file(dicom_loc, force=True)
    
    # get patient, study, and series information
    patientID = clean_text(ds.get("PatientID", "NA"))
    studyDate = clean_text(ds.get("StudyDate", "NA"))
    studyDescription = clean_text(ds.get("StudyDescription", "NA"))
    seriesDescription = clean_text(ds.get("SeriesDescription", "NA"))
#     print(type(ds[0x0008, 0x103e])) #<class 'pydicom.dataelem.DataElement'>
    # generate new, standardized file name
    modality = ds.get("Modality","NA")
    studyInstanceUID = ds.get("StudyInstanceUID","NA")
#     seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
    instanceNumber = str(ds.get("InstanceNumber","0"))
    fileName = modality + "." + seriesDescription + "." + instanceNumber
#     fileName = modality + "." + seriesDescription + "." + instanceNumber

    # save files to a 4-tier nested folder structure
    if not os.path.exists(os.path.join(dst, patientID)):
            os.makedirs(os.path.join(dst, patientID))
    if not os.path.exists(os.path.join(dst, patientID, studyDate)):
            os.makedirs(os.path.join(dst, patientID, studyDate))
    if not os.path.exists(os.path.join(dst, patientID, studyDate, studyDescription)):
            os.makedirs(os.path.join(dst, patientID, studyDate, studyDescription))
    if not os.path.exists(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription)):
            os.makedirs(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription))
            print('Saving out file: %s - %s - %s - %s.' % (patientID, studyDate, studyDescription, seriesDescription ))
    ds.save_as(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription, fileName))

print('done.')
    

Saving out file: anonn6rrpe1nm - 20140528 - pet_und_mrt_thorax_bis_becken_nativ_und_mit_k - becken_cor.
done.


In [47]:
# def search_file(directory = None, file = None):
#     assert os.path.isdir(directory)
#     for cur_path, directories, files in os.walk(directory):
#         if file in files:
#             return os.path.join(directory, cur_path, file)
#     return None
    

In [80]:
# folder = "/mnt/HDD1/shounak/test"
folder ="/mnt/HDD1/shounak/Biopsy_output_year_sorted/17"

# sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

# print(sub_folders)

# for root, dirs, files in os.walk(folder, topdown=False):
#     for name in dirs:
#         print (os.path.join(root, name))

# listdir = os.listdir(folder)
# print(listdir)

# for item in folder:

#     if os.path.isdir(item):

#         print(item)

# print (glob.glob("/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/*"))
# print(search_file("/mnt/HDD1/shounak/test", None))
dest = "/mnt/HDD1/shounak/test/testforcopyingfiles/"

for name in glob.glob('/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/*/*/*/prostate_diff_b800/'):
#     print ('\t', type(name)) #<class 'str'>
#     print ('\t', (name))
    path_src = pathlib.Path(name)
#     print(type(path_src))
    full_file_name = os.path.join(path_src, file_name)
    print(full_file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, dest)
#         src_files = os.listdir(path_src)
#     print(src_files)
# print("done")


/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anono14cff1mn/20170901/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anondphhef1rn/20160502/mrt_becken_nativ_und_mit_km/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anon9u3cff1ci/20170731/mrt_becken_nativ_und_mit_km/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anoni7hhef167/20161219/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anonff3cff1rj/20170901/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anondg5cff113/20170808/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.510

In [ ]:
opfolder = "/mnt/HDD1/shounak/Biopsy_output"
filecount = []  #empty array which will store the file names
modalitycount = []
for root, dirs, files in os.walk(opfolder):
    for file in files: 
#         if ".file" in file:# exclude non-dicoms, good for messy folders
            filecount.append(os.path.join(root, file))
            for dicom_loc in filecount:
    # read the file
                ds1 = dicom.read_file(dicom_loc, force=True)
    
    # get patient, study, and series information
                modalitycount = ds1.get("Modality","NA")
#                 modalitycount.append(os.path.join(root, file))
print('%s files found.' % len(filecount))
print('modality count=', len(modalitycount))

In [ ]:
patientidlist = []
for root, dirs, files in os.walk(patientidlist):
    for file in files: 
#         if ".file" in file:# exclude non-dicoms, good for messy folders
            filecount.append(os.path.join(root, file))

print('%s files found.' % len(patientidlist))

for dicom_loc in filecount:
    # read the file
    ds1 = dicom.read_file(dicom_loc, force=True)
    
    # get patient, study, and series information
    mod = ds1.get("Modality","NA")
#     studyInstanceUID = ds1.get("StudyInstanceUID","NA")
#     seriesInstanceUID = ds1.get("SeriesInstanceUID","NA")
#     instanceNumber = str(ds1.get("InstanceNumber","0"))
    
    patientidlist.append(os.path.join(root, files))